In [30]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
from torchsummary import summary
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [31]:
data = pd.read_csv('./BP_data.csv')

In [32]:
data

,Patient_Number,Blood_Pressure_Abnormality,Level_of_Hemoglobin,Genetic_Pedigree_Coefficient,Age,BMI,Sex,Pregnancy,Smoking,Physical_activity,salt_content_in_the_diet,alcohol_consumption_per_day,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders
0,1,1,11.28,0.90,34,23,1,1.0,0,45961,48071,NaN,2,1,1
1,2,0,9.75,0.23,54,33,1,NaN,0,26106,25333,205.0,3,0,0
2,3,1,10.79,0.91,70,49,0,NaN,0,9995,29465,67.0,2,1,0
3,4,0,11.00,0.43,71,50,0,NaN,0,10635,7439,242.0,1,1,0
4,5,1,14.17,0.83,52,19,0,NaN,0,15619,49644,397.0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,1,10.14,0.02,69,26,1,NaN,1,26118,47568,144.0,3,1,0
1996,1997,1,11.77,1.00,24,45,1,1.0,1,2572,8063,NaN,3,1,1
1997,1998,1,16.91,0.22,18,42,0,NaN,0,14933,24753,NaN,2,1,1
1998,1999,0,11.15,0.72,46,45,1,NaN,1,18157,15275,253.0,3,0,1


In [33]:
print(data.isnull().sum())  

Patient_Number                      0
Blood_Pressure_Abnormality          0
Level_of_Hemoglobin                 0
Genetic_Pedigree_Coefficient       92
Age                                 0
BMI                                 0
Sex                                 0
Pregnancy                        1558
Smoking                             0
Physical_activity                   0
salt_content_in_the_diet            0
alcohol_consumption_per_day       242
Level_of_Stress                     0
Chronic_kidney_disease              0
Adrenal_and_thyroid_disorders       0
dtype: int64


In [34]:
# Genetic_Pedigree_Coefficient: 중앙값으로 대체
data['Genetic_Pedigree_Coefficient'].fillna(data['Genetic_Pedigree_Coefficient'].median(), inplace=True)

# Pregnancy: 성별 기준으로 나눠서 결측 처리하거나, 컬럼 자체 제거 고려
data.drop('Pregnancy', axis=1, inplace=True)  # 예시

data['alcohol_consumption_per_day'].fillna(data['alcohol_consumption_per_day'].median(), inplace=True)


C:\Users\syjwk\AppData\Local\Temp\ipykernel_16648\3694012886.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Genetic_Pedigree_Coefficient'].fillna(data['Genetic_Pedigree_Coefficient'].median(), inplace=True)
C:\Users\syjwk\AppData\Local\Temp\ipykernel_16648\3694012886.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

In [35]:
print(data.isnull().sum())  

Patient_Number                   0
Blood_Pressure_Abnormality       0
Level_of_Hemoglobin              0
Genetic_Pedigree_Coefficient     0
Age                              0
BMI                              0
Sex                              0
Smoking                          0
Physical_activity                0
salt_content_in_the_diet         0
alcohol_consumption_per_day      0
Level_of_Stress                  0
Chronic_kidney_disease           0
Adrenal_and_thyroid_disorders    0
dtype: int64


In [36]:
# 데이터와 타겟 분리
X = data.drop(['Blood_Pressure_Abnormality', 'Patient_Number'], axis=1).values
y = data['Blood_Pressure_Abnormality'].values

In [37]:
# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 훈련 데이터, 테스트 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [38]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, 

((1600, 12), (400, 12), (1600,), (400,))

In [39]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [40]:
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 입력값, 정답값 하나로 묶기
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [41]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([1600, 12]),
 torch.Size([400, 12]),
 torch.Size([1600]),
 torch.Size([400]))

In [ ]:
class DiabetesClassifier(nn.Module):
    def __init__(self):
        super(DiabetesClassifier, self).__init__()
        self.fc1 = nn.Linear(12, 64)     # 입력특성: 15개, 노드수: 64개
        self.fc2 = nn.Linear(64, 32)    # 노드수: 64 -> 32
        self.fc3 = nn.Linear(32, 2)  # 노드수: 32 -> 2(클래스 개수)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = DiabetesClassifier()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Epoch 1/20, Loss: 0.2575, Accuracy: 1644.00%
Epoch 2/20, Loss: 0.2583, Accuracy: 1644.00%
Epoch 3/20, Loss: 0.2532, Accuracy: 1644.00%
Epoch 4/20, Loss: 0.2497, Accuracy: 1644.00%
Epoch 5/20, Loss: 0.2446, Accuracy: 1644.00%
Epoch 6/20, Loss: 0.2422, Accuracy: 1644.00%
Epoch 7/20, Loss: 0.2439, Accuracy: 1644.00%
Epoch 8/20, Loss: 0.2383, Accuracy: 1644.00%
Epoch 9/20, Loss: 0.2360, Accuracy: 1644.00%
Epoch 10/20, Loss: 0.2322, Accuracy: 1644.00%
Epoch 11/20, Loss: 0.2294, Accuracy: 1644.00%
Epoch 12/20, Loss: 0.2303, Accuracy: 1644.00%
Epoch 13/20, Loss: 0.2250, Accuracy: 1644.00%
Epoch 14/20, Loss: 0.2228, Accuracy: 1644.00%
Epoch 15/20, Loss: 0.2205, Accuracy: 1644.00%
Epoch 16/20, Loss: 0.2174, Accuracy: 1644.00%
Epoch 17/20, Loss: 0.2133, Accuracy: 1644.00%
Epoch 18/20, Loss: 0.2137, Accuracy: 1644.00%
Epoch 19/20, Loss: 0.2128, Accuracy: 1644.00%
Epoch 20/20, Loss: 0.2110, Accuracy: 1644.00%
Training complete.


In [29]:
# 학습 중 손실값과 정확도 저장할 리스트
train_losses = []
test_accuracies = []

# 학습 반복 횟수 설정 (에폭 수)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0

    # 훈련용 데이터를 32개씩 나누어 학습
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()         # 매번 기울기 초기화
        outputs = model(inputs)       # 예측값 계산 (순전파)
        loss = criterion(outputs, labels)  # 예측값과 정답 비교해서 손실 계산
        loss.backward()               # 역전파: 손실을 기준으로 기울기 계산
        optimizer.step()              # 계산된 기울기로 가중치 업데이트

        running_loss += loss.item()   # 손실 누적

    # 한 에폭당 평균 손실 저장
    train_losses.append(running_loss / len(train_dataloader))

    # 테스트 데이터로 성능 평가
    model.eval()      # 평가 모드 전환
    correct = 0
    total = 0
    with torch.no_grad():  # 평가할 땐 기울기 계산 X (속도 ↑)
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # 예측 확률이 가장 높은 클래스 선택
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 정확도 계산 (올바르게 예측한 수 / 전체 수)
    accuracy = 100 * correct / total
    test_accuracies.append(accuracy)

    # 에폭별 결과 출력
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_losses[-1]:.4f}, Accuracy: {accuracy:.2f}%")

print("Training complete.")

Epoch 1/20, Loss: 0.2526, Accuracy: 1644.00%
Epoch 2/20, Loss: 0.2491, Accuracy: 1644.00%
Epoch 3/20, Loss: 0.2467, Accuracy: 1644.00%
Epoch 4/20, Loss: 0.2417, Accuracy: 1644.00%
Epoch 5/20, Loss: 0.2389, Accuracy: 1644.00%
Epoch 6/20, Loss: 0.2386, Accuracy: 1644.00%
Epoch 7/20, Loss: 0.2363, Accuracy: 1644.00%
Epoch 8/20, Loss: 0.2321, Accuracy: 1644.00%
Epoch 9/20, Loss: 0.2288, Accuracy: 1644.00%
Epoch 10/20, Loss: 0.2283, Accuracy: 1644.00%
Epoch 11/20, Loss: 0.2217, Accuracy: 1644.00%
Epoch 12/20, Loss: 0.2229, Accuracy: 1644.00%
Epoch 13/20, Loss: 0.2206, Accuracy: 1644.00%
Epoch 14/20, Loss: 0.2171, Accuracy: 1644.00%
Epoch 15/20, Loss: 0.2119, Accuracy: 1644.00%
Epoch 16/20, Loss: 0.2109, Accuracy: 1644.00%
Epoch 17/20, Loss: 0.2084, Accuracy: 1644.00%
Epoch 18/20, Loss: 0.2076, Accuracy: 1644.00%
Epoch 19/20, Loss: 0.2037, Accuracy: 1644.00%
Epoch 20/20, Loss: 0.2005, Accuracy: 1644.00%
Training complete.
